In [ ]:
from cgitb import text
from traceback import print_tb
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from curses.textpad import Textbox
import string
from time import strftime, time
from tkinter import*
import tkinter as tk
from tkinter.ttk import Progressbar
from PIL import Image,ImageTk,ImageDraw
from tkinter import messagebox
from datetime import datetime
from time import strftime
from pytz import timezone
from gtts import gTTS 
import random
from IPython.display import Audio 
import pygame
pygame.mixer.init()
import time
root=tk.Tk()
root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(0, weight=1)
root.geometry("1550x800+0+0")
root['bg']='black'
root.title("Youtube-Transcript Summarizer")

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")
#========================================= time ==========================================================#
def current_time():
    now_utc = datetime.now(timezone('UTC'))
    now_asia = now_utc.astimezone(timezone('Asia/Kolkata'))
    string=now_asia.strftime('%H:%M:%S %p')
    lbl.config(text=string)
    lbl.after(1000,current_time)

lbl=Label(root,font=(SUNKEN,15,'bold'),fg='white',bg='black',border=0,borderwidth=0)
lbl.place(x=1,y=5)
current_time()
        
#======================================= Heading ========================================================#
img1=Image.open(r"image_resource\you_img.jpg")
img1=img1.resize((130,120),Image.ANTIALIAS)
img1=ImageTk.PhotoImage(img1)

hd_img1=Label(root,image=img1,bg='black')
hd_img1.place(x=1040,y=3.5)
hd_text=Label(root,text="Youtube Transcript Summarizer",font=(SUNKEN,30,'bold'),fg='white',bg='black',border=0,borderwidth=0)
hd_text.place(x=450,y=30)

#======================================== Exit Button ===================================================#
def exit_fun():
    root.destroy()
def iExit():
    iExit_t=messagebox.askyesno("Exit","Do you want to exit")
    if iExit_t>0:
        root.destroy()
    else:
        return    
exit_btn=Button(root,text='Quit',bg='red',command=iExit,font=('Arial',13,'bold'),fg='white',activebackground='red',activeforeground='white',border=0,borderwidth=3,cursor='hand2',relief=RIDGE)
exit_btn.place(x=1400,y=10,width=100,height=25)


#====================================== get-started & stop ==============================================#
def getlink():
    link=StringVar()
    
    def var_text():
        name=link.get()
        with open("link.txt", "w") as text_file:
            text_file.write(name)
        link.set("")
        
        progress = Progressbar(root, orient = HORIZONTAL,length = 200, mode = 'determinate')
        def youtube():
            progress['value'] = 40
            root.update_idletasks()
            time.sleep(0.5)
            myfile = open('link.txt', "r")
            you_link = myfile.readline()
            myfile.close()
            video_id = you_link.split("=")[1]
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            full_transcript = ""
            for x in transcript:
                full_transcript += " " + x['text']

            print(len(full_transcript))
            
            progress['value'] = 80
            root.update_idletasks()
            time.sleep(0.5)
            
            summerizar = pipeline("summarization")
            itr = int(len(full_transcript)/400)
            summary_text = []
            
            progress['value'] = 120
            root.update_idletasks()
            time.sleep(0.1)
            
            for i in range(0, itr+1):
                start = 0
                start = i*400
    
                end = (i+1)*400
                out = summerizar(full_transcript[start:end], max_length=57)
                out = out[0]
                out = out["summary_text"]
                summary_text.append(out)
                
            progress['value'] = 160
            root.update_idletasks()
            time.sleep(0.1)    
            
            with open("summary.txt", "w") as text_file:
                text_file.write(summary_text[0])
                
            progress['value'] = 200
            root.update_idletasks()
            time.sleep(5) 
        progress.place(x=1150,y=245)    
        transcribe=Button(root,text='Transcribe',bg='green',command=youtube,font=('Arial',13,'bold'),fg='white',activebackground='red',activeforeground='white',border=0,borderwidth=3,cursor='hand2',relief=RIDGE)
        transcribe.place(x=1000,y=240,width=137,height=32)

    link_lbl=Label(root,text="Enter URL",bg='black',fg='light blue',font=('SUNKEN',13,'bold'))
    link_lbl.place(x=100,y=250)
    link_data=tk.Entry(root,width=20,textvariable=link,font=('SUNKEN',13,'normal'))
    link_data.place(x=200,y=250)
    
    sub_btn=Button(root,text="Submit",command=var_text,bg='white',fg='black',border=1,borderwidth=3,font=('Arial',10),activebackground='green')
    sub_btn.place(x=400,y=250,height=25)
    
    
    
    
start_img=Image.open(r"image_resource\button_activate.png")
start_img=start_img.resize((204,55),Image.ANTIALIAS)
start_img=ImageTk.PhotoImage(start_img)
start=Button(root,command=getlink,image=start_img,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
start.place(x=670,y=160,width=207,height=57)


#========================================= text-area =====================================================#
txtarea = Text(root,wrap=WORD,bg='lavender',font=("TimesNewRoman",11),width=90, height=20,borderwidth=4,relief='solid',padx=10,pady=10,spacing2=5)
txtarea.pack(pady=100)
txtarea.place(x=400,y=320)
#======================================= Get Link & API call ======================================================#

def english():
    txtarea.delete('1.0', END)
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    txtarea.delete('1.0', END)
    txtarea.insert(tk.END, data )
    
    
    pygame.mixer.init()
    r1 = random.randint(1,10000000)
    r2 = random.randint(1,10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) +".mp3"
    file = open('summary.txt',"r")
    data = file.readline()
    file.close()
    text = data
    language = 'en'
    speech = gTTS(text = text, lang = language, slow = False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)
    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    def stop_sound():
        pygame.mixer.music.stop()
        
    play=Button(root,text="Play",command=play_sound,bg='black',fg='green',cursor='hand2',border=2,borderwidth=2,activebackground='black',font=('SUNKEN',13,'bold'))
    play.place(x=350,y=350,width=40,height=20)  
   
def hindi():
    txtarea.delete('1.0', END)                                             
    file = open('summary.txt',"r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
         forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
    )
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(translation)
    
    txtarea.insert(tk.END, translation[0] )
    text = translation[0]
    language = 'hi'
    speech = gTTS(text = text, lang = language, slow = False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)
    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    def stop_sound():
        pygame.mixer.music.stop()
        
    play=Button(root,text="Play",command=play_sound,bg='black',fg='green',cursor='hand2',border=2,borderwidth=2,activebackground='black',font=('SUNKEN',13,'bold'))
    play.place(x=350,y=450,width=40,height=20)
    
def german():
    txtarea.delete('1.0', END)                                             
    file = open('summary.txt',"r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
         forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"]
    )
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0] )
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1,10000000)
    r2 = random.randint(1,10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) +".mp3"
    language = 'de'
    speech = gTTS(text = text, lang = language, slow = False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)
    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    def stop_sound():
        pygame.mixer.music.stop()
        
    play=Button(root,text="Play",command=play_sound,bg='black',fg='green',cursor='hand2',border=2,borderwidth=2,activebackground='black',font=('SUNKEN',13,'bold'))
    play.place(x=350,y=550,width=40,height=20)
def french():
    txtarea.delete('1.0', END)                                             
    file = open('summary.txt',"r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
         forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"]
    )
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0] )
    pygame.mixer.init()
    r1 = random.randint(1,10000000)
    r2 = random.randint(1,10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) +".mp3"
    text = translation[0]
    language = 'fr'
    speech = gTTS(text = text, lang = language, slow = False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)
    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    
    play=Button(root,text="Play",command=play_sound,bg='black',fg='green',cursor='hand2',border=2,borderwidth=2,activebackground='black',font=('SUNKEN',13,'bold'))
    play.place(x=350,y=650,width=40,height=20)
def bengali():
    txtarea.delete('1.0', END)                                             
    file = open('summary.txt',"r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
         forced_bos_token_id=tokenizer.lang_code_to_id["bn_IN"]
    )
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0] )
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1,10000000)
    r2 = random.randint(1,10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) +".mp3"
    language = 'bn'
    speech = gTTS(text = text, lang = language, slow = False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)
    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    play=Button(root,text="Play",command=play_sound,bg='black',fg='green',cursor='hand2',border=2,borderwidth=2,activebackground='black',font=('SUNKEN',13,'bold'))
    play.place(x=1160,y=350,width=40,height=20)
    
def chinese():
    txtarea.delete('1.0', END)                                             
    file = open('summary.txt',"r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
         forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
    )
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0] )
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1,10000000)
    r2 = random.randint(1,10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) +".mp3"
    language = 'zh-cn'
    speech = gTTS(text = text, lang = language, slow = False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)
    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    play=Button(root,text="Play",command=play_sound,bg='black',fg='green',cursor='hand2',border=2,borderwidth=2,activebackground='black',font=('SUNKEN',13,'bold'))
    play.place(x=1160,y=450,width=40,height=20)
    
def tamil():
    txtarea.delete('1.0', END)                                             
    file = open('summary.txt',"r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
         forced_bos_token_id=tokenizer.lang_code_to_id["ta_IN"]
    )
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0] )
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1,10000000)
    r2 = random.randint(1,10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) +".mp3"
    language = 'ta'
    speech = gTTS(text = text, lang = language, slow = False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)
    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    play=Button(root,text="Play",command=play_sound,bg='black',fg='green',cursor='hand2',border=2,borderwidth=2,activebackground='black',font=('SUNKEN',13,'bold'))
    play.place(x=1160,y=550,width=40,height=20)
    
def telugu():
    txtarea.delete('1.0', END)                                             
    file = open('summary.txt',"r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
         forced_bos_token_id=tokenizer.lang_code_to_id["te_IN"]
    )
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0] )
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1,10000000)
    r2 = random.randint(1,10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) +".mp3"
    language = 'te'
    speech = gTTS(text = text, lang = language, slow = False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)
    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    play=Button(root,text="Play",command=play_sound,bg='black',fg='green',cursor='hand2',border=2,borderwidth=2,activebackground='black',font=('SUNKEN',13,'bold'))
    play.place(x=1160,y=650,width=40,height=20)
    
def stop_sound():
    pygame.mixer.music.stop()
stop_img=Image.open(r"image_resource\stop_button.png")
stop_img=stop_img.resize((40,38),Image.ANTIALIAS)
stop_img=ImageTk.PhotoImage(stop_img)
stop=Button(root,image=stop_img,command=stop_sound,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
stop.place(x=740,y=700,width=50,height=40)     
#======================================= Language-button ===============================================#


btn_eng=Image.open(r"image_resource\button_english.png")
btn_eng=btn_eng.resize((132,34),Image.ANTIALIAS)
btn_eng=ImageTk.PhotoImage(btn_eng)
eng=Button(root,image=btn_eng,command=english,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
eng.place(x=200,y=340,width=135,height=37)

btn_hindi=Image.open(r"image_resource\button_hindi.png")
btn_hindi=btn_hindi.resize((132,34),Image.ANTIALIAS)
btn_hindi=ImageTk.PhotoImage(btn_hindi)
hindi=Button(root,image=btn_hindi,command=hindi,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
hindi.place(x=200,y=440,width=135,height=37)

btn_german=Image.open(r"image_resource\button_german.png")
btn_german=btn_german.resize((132,34),Image.ANTIALIAS)
btn_german=ImageTk.PhotoImage(btn_german)
german=Button(root,image=btn_german,command=german,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
german.place(x=200,y=540,width=135,height=37)

btn_french=Image.open(r"image_resource\button_french.png")
btn_french=btn_french.resize((132,34),Image.ANTIALIAS)
btn_french=ImageTk.PhotoImage(btn_french)
french=Button(root,image=btn_french,command=french,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
french.place(x=200,y=640,width=135,height=37)

btn_bengali=Image.open(r"image_resource\button_bengali.png")
btn_bengali=btn_bengali.resize((132,34),Image.ANTIALIAS)
btn_bengali=ImageTk.PhotoImage(btn_bengali)
bengali=Button(root,image=btn_bengali,command=bengali,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
bengali.place(x=1210,y=340,width=135,height=37)

btn_chinise=Image.open(r"image_resource\button_chinese.png")
btn_chinise=btn_chinise.resize((132,34),Image.ANTIALIAS)
btn_chinise=ImageTk.PhotoImage(btn_chinise)
chinise=Button(root,image=btn_chinise,command=chinese,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
chinise.place(x=1210,y=440,width=135,height=37)

btn_tamil=Image.open(r"image_resource\button_tamil.png")
btn_tamil=btn_tamil.resize((132,34),Image.ANTIALIAS)
btn_tamil=ImageTk.PhotoImage(btn_tamil)
tamil=Button(root,image=btn_tamil,command=tamil,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
tamil.place(x=1210,y=540,width=135,height=37)

btn_telugu=Image.open(r"image_resource\button_telugu.png")
btn_telugu=btn_telugu.resize((132,34),Image.ANTIALIAS)
btn_telugu=ImageTk.PhotoImage(btn_telugu)
telugu=Button(root,image=btn_telugu,command=telugu,bg='black',fg='red',cursor='hand2',border=0,borderwidth=0,activebackground='black')
telugu.place(x=1210,y=640,width=135,height=37)


root.mainloop()  

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
476


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 57, but you input_length is only 28. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
C:\Users\hp\anaconda3\lib\site-packages\transformers\generation\utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['डैगन स्क्रॉल की शक्ति है.. मेरा! यह कुछ नहीं है! मुझे इसे पहली बार भी नहीं मिला था. वहाँ कोई गुप्त घटक नहीं है. यह सिर्फ आप हैं. मैं चीखने जा रहा हूँ! मत करें! मत करें! आप सिर्फ']


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-1-2b8daefbfc4d>", line 201, in hindi
    speech.save(sound_file)
NameError: name 'sound_file' is not defined


['Die Macht der Drachenrolle ist.. Mein! Es ist nichts! Ich habe es auch nicht das erste Mal bekommen. Es gibt keinen geheimen Inhaltsstoff. Es ist nur Sie. Ich werde Pee! Nicht! Nicht! Sie sind nur']


In [2]:
!git status

On branch main
Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    1004744SUMMARY3918359.mp3
	deleted:    1004788SUMMARY8760205.mp3
	deleted:    1065636SUMMARY4287325.mp3
	deleted:    1362765SUMMARY7449454.mp3
	deleted:    1429269SUMMARY7959864.mp3
	deleted:    1754339SUMMARY1611747.mp3
	deleted:    1972440SUMMARY4558853.mp3
	deleted:    2038298SUMMARY4952204.mp3
	deleted:    2650472SUMMARY8781926.mp3
	deleted:    2949467SUMMARY7652338.mp3
	deleted:    3186208SUMMARY8220541.mp3
	deleted:    3782817SUMMARY281720.mp3
	deleted:    3791114SUMMARY9451974.mp3
	deleted:    4259378SUMMARY2308448.mp3
	deleted:    5200836SUMMARY8586780.mp3
	deleted:    5263056SUMMARY6921871.mp3
	deleted:    5924407SUMMARY635625.mp3
	deleted:    7088634SUMMARY6879125.mp3
	modified:   8041297SUMMARY4146643.mp3
	deleted:    8160754SUMMARY23615.mp3
	deleted:    8289807SUMMARY9816585.mp3
	de

In [55]:
!git push -u  https://github.com/ups1610/Youtube-Transcript.git

Branch 'main' set up to track remote branch 'main' from 'https://github.com/ups1610/Youtube-Transcript.git'.


To https://github.com/ups1610/Youtube-Transcript.git
   1d0b6a8..534f5b6  main -> main
